# Detect leakage based on embedding similarity

We use the [Annoy library](https://github.com/spotify/annoy) to find nearest neighbors and to detect leakage in the test split.

More information about this play can be found in the Spotlight documentation: [Detect and remove leakage](https://renumics.com/docs/playbook/leakage_annoy)

For more data-centric AI workflows, check out our [Awesome Open Data-centric AI](https://github.com/Renumics/awesome-open-data-centric-ai) list on Github.

## tldr

In [ ]:
# @title Install required packages with PIP

!pip install renumics-spotlight datasets annoy

In [ ]:
# @title Play as copy-n-paste functions

import datasets
from renumics import spotlight
from annoy import AnnoyIndex
import pandas as pd
import requests
import json


def nearest_neighbor_annoy(
    df, embedding_name="embedding", threshold=0.3, tree_size=100
):
    embs = df[embedding_name]

    t = AnnoyIndex(len(embs[0]), "angular")

    for idx, x in enumerate(embs):
        t.add_item(idx, x)

    t.build(tree_size)

    images = df["image"]

    df_nn = pd.DataFrame()

    nn_id = [t.get_nns_by_item(i, 2)[1] for i in range(len(embs))]
    df_nn["nn_id"] = nn_id
    df_nn["nn_image"] = [images[i] for i in nn_id]
    df_nn["nn_distance"] = [t.get_distance(i, nn_id[i]) for i in range(len(embs))]
    df_nn["nn_flag"] = df_nn.nn_distance < threshold

    return df_nn


def detect_leakage(df, nn_id_name="nn_id", split_name="split", threshold=0.3):
    nn_id = df["nn_id"]
    split = split = df["split"]

    df_leak = pd.DataFrame()
    df_leak["nn_same_split"] = [
        True if split[i] == split[nn_id[i]] else False for i in range(len(nn_id))
    ]
    df_leak["leakage_flag"] = (df.nn_distance < 0.3) & (
        df_leak["nn_same_split"] != True
    )

    return df_leak

## Step-by-step example on CIFAR-100

### Load CIFAR-100 from Huggingface hub and convert it to Pandas dataframe

In [ ]:
dataset = datasets.load_dataset("renumics/cifar100-enriched", split="all")

df = dataset.to_pandas()

### Compute heuristics for data leakage based on nearest neighbor distance

In [ ]:
df_nn = nearest_neighbor_annoy(df)
df = pd.concat([df, df_nn], axis=1)
df_leak = detect_leakage(df)
df = pd.concat([df, df_leak], axis=1)

### Inspect and remove leakage with Spotlight

> ⚠️ Running Spotlight in Colab currently has severe limitations (slow, no similarity map, no layouts) due to Colab restrictions (e.g. no websocket support). Run the notebook locally for the full Spotlight experience.

In [ ]:
df_show = df.drop(columns=["embedding", "probabilities"])


# handle google colab differently
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # visualization in Google Colab only works in chrome and does not support websockets, we need some hacks to visualize something
    df_show = df_show[:10000]
    df_show["embx"] = [emb[0] for emb in df_show["embedding_reduced"]]
    df_show["emby"] = [emb[1] for emb in df_show["embedding_reduced"]]
    port = 50123
    layout_url = "https://raw.githubusercontent.com/Renumics/spotlight/main/playbook/veteran/leakage_annoy_colab.json"
    response = requests.get(layout_url)
    layout = spotlight.layout.nodes.Layout(**json.loads(response.text))
    spotlight.show(
        df_show,
        port=port,
        dtype={"image": spotlight.Image, "nn_image": spotlight.Image},
        layout=layout,
    )
    from google.colab.output import eval_js  # type: ignore

    print(str(eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': true}})")))

else:
    df_show = df.drop(columns=["embedding", "probabilities"])
    layout_url = "https://raw.githubusercontent.com/Renumics/spotlight/main/playbook/veteran/leakage_annoy.json"
    response = requests.get(layout_url)
    layout = spotlight.layout.nodes.Layout(**json.loads(response.text))
    spotlight.show(
        df_show,
        dtype={
            "image": spotlight.Image,
            "nn_image": spotlight.Image,
            "embedding_reduced": spotlight.Embedding,
        },
        layout=layout,
    )